# m-PDTSP

In a multi-commodity pick-and-delivery traveling salesperson problem (m-PDTSP), we are given the set of location $N = \{ 0, ..., n - 1 \}$ and the set of edges $A \subseteq N \times N$.
A vehicle with the capacity $q$ starts from $0$, visit all customers $\{ 1, ..., n - 2 \}$, and stops at $n - 1$.
Visiting customer $j$ from $i$ is possible only if $(i, j) \in A$ and incurs the travel cost $c_{ij}$.
There are commodities $M = \{ 0, ..., m - 1 \}$, and commodity $k \in M$ with weight $w_k$ is picked up at customer $p_k \in N$ and delivered to customer $d_k \in N$.
We want to find a tour to visit all customers while picking up and delivering all commodities and minimizing the total travel cost.

## DP Formulation

When the vehicle visits customer $j$, the net change of the load is

$$
    \delta_j = \sum_{k \in M : p_k = j} w_k - \sum_{k \in M : d_k = j} w_k.
$$

Because we need to pick up before delivery, the set of customers that must be visited before $j$ is

$$
    P_j = \{ p_k \mid k \in M : d_k = j \}.
$$

Let $U$ be the set of customers that are not visited yet, $i$ be the current location of the vehicle, and $l$ be the current load of the vehicle.
Then, the set of customers that can be visited next is

$$
    R(U, i, l) = \{ j \in U \mid (i, j) \in A \land l + \delta_j \leq q \land P_j \cap U = \emptyset \}.
$$

Let $V(U, i, l)$ be the minimum cost to visit all customers in $U$ from $i$ with the load $l$.
We have the following DP model:

$$
\begin{align}
    \text{compute } & V(N \setminus \{ 0, n - 1 \}, 0, 0) \\
    & V(U, i, l) = \begin{cases}
        \min\limits_{j \in R(U, i, l)} c_{ij} + V(U \setminus \{ j \}, j, l + \delta_j) & \text{if } U \neq \emptyset \\
        c_{i, n - 1} + V(U, n - 1, l) & \text{if } U = \emptyset \land (i, j) \in A \\
        0 & \text{if } U = \emptyset \land i = n - 1.
    \end{cases}
\end{align}
$$

When two states $(U, i, l)$ and $(U, i, l')$ have the same set of unvisited customers $U$ and the same location $i$, if $l \leq l'$, $(U, i, l)$ leads to a better solution. Threfore,

$$
    V(U, i, l) \leq V(U, i, l') \text{ if } l \leq l'.
$$

The lowest possible travel cost to visit customer $j$ is $\min_{k \in N : (k, j) \in A} c_{kj}$.
Because we need to visit all customers in $U$, the total travel cost is at least $\sum_{j \in U} \min_{k \in N : (k, j) \in A } c_{kj}$. Furthermore, if the current location $i$ is not $n + 1$, we need to visit $n - 1$.
Therefore,

$$
    V(U, i, k, l) \geq \sum_{j \in (U \cup \{ n - 1 \}) \setminus \{ i \} } \min_{k \in N : (k, j) \in A} c_{kj}.
$$

Similarly, we need to depart from each customer in $U$ and the current location $i$ if $i \neq n + 1$.
Therefore,

$$
    V(U, i, k, l) \geq \sum_{j \in (U \cup \{ i \}) \setminus \{ n - 1 \} } \min_{k \in N : (j, k) \in A} c_{jk}.
$$

## Install DIDPPy

In [1]:
!pip install didppy

## Data

In [2]:
# Number of locations
n = 5
# Edges
a = {
    (0, 1), (0, 2), (0, 3),
    (1, 2), (1, 3), (1, 4),
    (2, 1), (2, 3), (2, 4),
    (3, 1), (3, 2), (3, 4),
}
# Number of commidities
m = 2
# Capacity of the vehicle
q = 5
# Weights
w = [3, 4]
# Pick up points
p = [3, 2]
# Delivery points
d = [2, 1]
# Travel cost
c = [
    [0, 3, 4, 5, 0],
    [3, 0, 5, 4, 3],
    [4, 5, 0, 3, 4],
    [5, 4, 3, 0, 5],
    [0, 3, 4, 5, 0],
]

## Preprocessing

In [3]:
# Net change
delta = [0] * n
# Predecessors
capital_p = [set() for _ in range(n)]

for k in range(m):
    delta[p[k]] += w[k]
    delta[d[k]] -= w[k]
    capital_p[d[k]].add(p[k])

## Modeling

In [4]:
import didppy as dp

model = dp.Model()

customer = model.add_object_type(number=n)

# U
unvisited = model.add_set_var(object_type=customer, target=list(range(1, n - 1)))
# i
location = model.add_element_var(object_type=customer, target=0)
# l
load = model.add_int_resource_var(target=0, less_is_better=True)

connected = model.add_bool_table(
    [[(i, j) in a for j in range(n)] for i in range(n)]
)
predecessors = model.add_set_table(capital_p, object_type=customer)
distance = model.add_int_table(c)

for j in range(1, n):
    visit = dp.Transition(
        name="visit {}".format(j),
        cost=distance[location, j] + dp.IntExpr.state_cost(),
        effects=[
            (unvisited, unvisited.remove(j)),
            (location, j),
            (load, load + delta[j]),
        ],
        preconditions=[
            unvisited.contains(j),
            connected[location, j],
            load + delta[j] <= q,
            unvisited.isdisjoint(predecessors[j]),
        ],
    )
    model.add_transition(visit)

finish = dp.Transition(
    name="finish",
    cost=distance[location, n - 1] + dp.IntExpr.state_cost(),
    effects=[(location, n - 1)],
    preconditions=[unvisited.is_empty(), connected[location, n - 1]],
)
model.add_transition(finish)

model.add_base_case([unvisited.is_empty(), location == n - 1])

min_distance_to = model.add_int_table(
    [0] + [min(c[k][j] for k in range(n) if (k, j) in a) for j in range(1, n)]
)
model.add_dual_bound(
    min_distance_to[unvisited]
    + (location != n - 1).if_then_else(min_distance_to[n - 1], 0)
)

min_distance_from = model.add_int_table(
    [min(c[j][k] for k in range(n) if (j, k) in a) for j in range(n - 1)] + [0]
)
model.add_dual_bound(min_distance_from[unvisited] + min_distance_from[location])

## Solving

In [5]:
solver = dp.CABS(model, quiet=True)
solution = solver.search()

print("Transitions to apply:")
print("")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))

Transitions to apply:

visit 3
visit 2
visit 1
finish

Cost: 16
